In [9]:
import dask.dataframe as dd
import plotly.express as px
import os

In [10]:
# --- CONSTANTES DE COLUNAS
COL_DATA = "Data"
COL_TEMP_MAX = "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)"
COL_TEMP_MIN = "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)"
COL_PREC = "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"
COL_STATE = "state"
COL_REGION = "regiao"

# Caminho para Amostra
PATH = os.path.join("../datasample/sample.csv")
MAPPING_REGION = {
    "N":        "norte",
    "NE":    "nordeste",
    "CO": "centro-oeste",
    "SE":    "sudeste",
    "S":        "sul",
}

In [11]:
def load_and_concat(PATH, MAPPING_REGION):
    dfs = []
   
    ddf = dd.read_csv(
        PATH,
        assume_missing=True,
        parse_dates=[COL_DATA],
        low_memory=False
    )
    ddf = ddf.rename(columns={"region": "regiao"})
    ddf["regiao"] = ddf["regiao"].map(MAPPING_REGION)
    return ddf

def clean_data(df, cols_to_clean, state_col=None):
    for col in cols_to_clean:
        df = df.astype({col: "float64"})
    df = df.dropna(subset=cols_to_clean)
    cond = True
    for col in cols_to_clean:
        cond = cond & (df[col] != -9999)
    df = df[cond]
    if state_col:
        df = df.dropna(subset=[state_col])
    return df

def add_time_columns(df):
    df["ano"] = df[COL_DATA].dt.year
    df["mes"] = df[COL_DATA].dt.month
    return df

def get_estacao(mes):
    if mes in [12, 1, 2]:
        return "Verão"
    elif mes in [3, 4, 5]:
        return "Outono"
    elif mes in [6, 7, 8]:
        return "Inverno"
    else:
        return "Primavera"

def add_estacao_column(df):
    df["estacao"] = df["mes"].map(get_estacao)
    return df

def aggregate_annual_precipitation_by_region(df):
    agg = df.groupby(["regiao", "ano"])[COL_PREC].sum().rename("precipitacao_total_anual")
    return agg.compute().reset_index()

def aggregate_annual_temperature_by_region(df):
    agg = df.groupby(["regiao", "ano"])[[COL_TEMP_MAX]].agg({
        COL_TEMP_MAX: "mean"
    }).rename(columns={
        COL_TEMP_MAX: "temp_media"
    })
    return agg.compute().reset_index()

def aggregate_by_state(df):
    agg = df.groupby(COL_STATE)[
        [COL_TEMP_MAX, COL_TEMP_MIN, COL_PREC]
    ].agg({
        COL_TEMP_MAX: "mean",
        COL_TEMP_MIN: "mean",
        COL_PREC: "sum"
    }).rename(columns={
        COL_TEMP_MAX: "temp_max_media",
        COL_TEMP_MIN: "temp_min_media",
        COL_PREC: "precipitacao_total"
    })
    return agg.compute().reset_index()

def aggregate_by_season(df):
    agg = df.groupby("estacao")[[COL_TEMP_MAX, COL_TEMP_MIN, COL_PREC]].agg({
        COL_TEMP_MAX: "mean",
        COL_TEMP_MIN: "mean",
        COL_PREC: "sum"
    }).rename(columns={
        COL_TEMP_MAX: "temp_max_media",
        COL_TEMP_MIN: "temp_min_media",
        COL_PREC: "precipitacao_total"
    })
    return agg.compute().reset_index()

def plot_bar(df, x, y_vars, title=None, width=1200, height=600):
    df_melted = df.melt(id_vars=[x], value_vars=y_vars)
    fig = px.bar(
        df_melted,
        x=x,
        y="value",
        color="variable",
        barmode="group",
        labels={
            x: x.capitalize(),
            "value": "Valor",
            "variable": "Métrica"
        },
        title=title
    )
    fig.update_layout(
        width=width,
        height=height,
        bargap=0.2,
        legend_title="Métrica"
    )
    fig.write_html(f"../misc/{(title.replace(',', '').lower()).replace(' ', '_')}.html")

In [12]:
df = load_and_concat(PATH, MAPPING_REGION)
df = clean_data(df, [COL_TEMP_MAX, COL_TEMP_MIN, COL_PREC], state_col=COL_STATE)
df = add_time_columns(df)
df = add_estacao_column(df)

# 1) Precipitação anual por região
annual_precip = aggregate_annual_precipitation_by_region(df)
fig = px.bar(
    annual_precip,
    x="ano",
    y="precipitacao_total_anual",
    color="regiao",
    barmode="group",
    labels={
        "ano": "Ano",
        "precipitacao_total_anual": "Precipitação Total Anual (mm)",
        "regiao": "Região"
    },
    title="Precipitação Total Anual por Região"
)
fig.update_layout(width=1200, height=500, bargap=0.2, legend_title="Região")
fig.write_html("../misc/grafico_precipitacao_anual_por_regiao.html")

# 2) Temperatura média anual por região
annual_temp = aggregate_annual_temperature_by_region(df)
fig = px.bar(
    annual_temp,
    x="ano",
    y="temp_media",
    color="regiao",
    barmode="group",
    labels={
        "ano": "Ano",
        "temp_media": "Temperatura Média (°C)",
        "regiao": "Região"
    },
    title="Temperatura Média Anual por Região"
)
fig.update_layout(width=1200, height=500, bargap=0.2, legend_title="Região")
fig.write_html("../misc/grafico_temperatura_media_anual_por_regiao.html")

# 3) Temperatura média e precipitação por estado
by_state = aggregate_by_state(df)
plot_bar(
    by_state,
    x="state",
    y_vars=["temp_max_media", "temp_min_media", "precipitacao_total"],
    title="Média Temperaturas e Precipitação Total por Estado (Todos os Anos)"
)

# 4) Temperatura e precipitação por estação do ano
by_season = aggregate_by_season(df)
plot_bar(
    by_season,
    x="estacao",
    y_vars=["temp_max_media", "temp_min_media", "precipitacao_total"],
    title="Temperaturas e Precipitação por Estação do Ano (Todos os Anos)",
    width=1100,
    height=550
)

/opt/conda/lib/python3.10/site-packages/dask/dataframe/dask_expr/_collection.py:4208: UserWarning:


You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map function that you are using.
  Before: .map(func)
  After:  .map(func, meta=('regiao', 'object'))


/opt/conda/lib/python3.10/site-packages/dask/dataframe/dask_expr/_collection.py:4208: UserWarning:


You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map function that you are using.
  Before: .map(func)
  After:  .map(func, meta=('mes', 'object'))


